## Overarching Team Data

In [1]:
import pandas as pd
import sqlite3

In [2]:
# prepare sql database
conn = sqlite3.connect('siege.db')
cursor = conn.cursor()

In [3]:
def import_data(name, drop_last = False): # imports data into python, cleans it, and imports into sql

    if name == "everything":
        global everything, players

        everything = pd.read_csv(f"C:\\Users\\aethe\\OneDrive\\Desktop\\stats\\{name}.csv")
        everything = everything.drop(["Season", "Warm_Up"], axis = 1)
        everything["Map"] = everything["Map"].str.replace(" ", "_")
        everything["Map"] = everything["Map"].str.lower()
        
        players = []
        column_names = list(everything.columns)
        for column in column_names:
            if "_score" in column:
                players.append(column.replace("_score", ""))
    # needs to be done with everything

        # observations in Result column need to be standardized as convention changed over time
        results = list(everything["Result"])
        results_new = []
        for i in range(len(results)):
            results_new.append(results[i].lower())
        everything["Result"] = results_new

        everything.columns = everything.columns.str.lower()
        
        # import into sql
        cursor.execute("drop table if exists everything")
        everything.to_sql("everything", conn, index = False)
    
    else:
        global season
        
        season = pd.read_csv(f"C:\\Users\\aethe\\OneDrive\\Desktop\\stats\\{name}.csv")
        season = season.drop(["season", "session", "date", "warm_up", "elo", "Δelo"], axis = 1)
        season["map"] = season["map"].str.replace(" ", "_")
        season["map"] = season["map"].str.lower()
        
        if drop_last == True:
            season = season.drop(season.tail(1).index)

        season.columns = season.columns.str.lower()
        
        cursor.execute("drop table if exists season")
        season.to_sql("season", conn, index = False)
    
    return everything

In [4]:
import_data("everything")
import_data("y9s2_games", drop_last = True) # change this depending on season & manual data cleanup

,date,game,map,squad_size,def_wins,def_losses,atk_wins,atk_losses,result,josh_score,...,p3_deaths,p3_assists,p4_score,p4_kills,p4_deaths,p4_assists,p5_score,p5_kills,p5_deaths,p5_assists
0,NaN,1,coastline,NaN,NaN,NaN,NaN,NaN,win,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,2,theme_park,NaN,NaN,NaN,NaN,NaN,loss,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,3,chalet,NaN,NaN,NaN,NaN,NaN,loss,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,4,villa,NaN,NaN,NaN,NaN,NaN,win,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,5,club_house,NaN,NaN,NaN,NaN,NaN,win,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
686,8/25/2024,35,outback,5.0,3.0,1.0,2.0,2.0,win,4965.0,...,6.0,2.0,4010.0,5.0,4.0,2.0,3900.0,4.0,6.0,3.0
687,8/26/2024,36,oregon,3.0,1.0,2.0,1.0,2.0,loss,1345.0,...,4.0,3.0,1265.0,3.0,5.0,1.0,1089.0,2.0,5.0,0.0
688,8/27/2024,37,club_house,4.0,2.0,2.0,3.0,1.0,win,4390.0,...,7.0,2.0,4390.0,7.0,6.0,0.0,3900.0,3.0,8.0,2.0
689,8/27/2024,38,border,5.0,1.0,1.0,3.0,0.0,win,4000.0,...,3.0,2.0,3490.0,2.0,3.0,2.0,3285.0,2.0,2.0,1.0


In [5]:
def get_sql_columns(name): # used for building the loop below
    cursor.execute(f"pragma table_info({name})")
    column_names = [desc[1] for desc in cursor.fetchall()]
    print(f"{name}: {column_names}")
    return

In [6]:
get_sql_columns("season")
get_sql_columns("everything")

season: ['game', 'map', 'squad_size', 'def_wins', 'def_losses', 'atk_wins', 'atk_losses', 'result', 'josh_score', 'josh_kills', 'josh_deaths', 'josh_assists', 'asa_score', 'asa_kills', 'asa_deaths', 'asa_assists', 'biggs_score', 'biggs_kills', 'biggs_deaths', 'biggs_assists', 'clark_score', 'clark_kills', 'clark_deaths', 'clark_assists', 'cole_score', 'cole_kills', 'cole_deaths', 'cole_assists', 'conell_score', 'conell_kills', 'conell_deaths', 'conell_assists', 'ed_score', 'ed_kills', 'ed_deaths', 'ed_assists', 'jakob_score', 'jakob_kills', 'jakob_deaths', 'jakob_assists', 'luke_score', 'luke_kills', 'luke_deaths', 'luke_assists', 'mike_score', 'mike_kills', 'mike_deaths', 'mike_assists', 'p1_score', 'p1_kills', 'p1_deaths', 'p1_assists', 'p2_score', 'p2_kills', 'p2_deaths', 'p2_assists', 'p3_score', 'p3_kills', 'p3_deaths', 'p3_assists', 'p4_score', 'p4_kills', 'p4_deaths', 'p4_assists', 'p5_score', 'p5_kills', 'p5_deaths', 'p5_assists']
everything: ['date', 'game', 'map', 'squad_size

In [7]:
def get_player_data(player_name, dataframe): # gets individual player score, kills, deaths, assists, games and results

    cursor.execute(f"drop table if exists {player_name}_{dataframe}")
    cursor.execute(f"create table {player_name}_{dataframe} as \
                   select game, map, atk_wins, def_wins, result, {player_name}_score, {player_name}_kills, {player_name}_deaths, {player_name}_assists from {dataframe}")
    cursor.execute(f"update {player_name}_{dataframe} set {player_name}_score = {player_name}_score - 2000 where result = 'win'")
    
    return

In [8]:
def extract_score_data(player_name, dataframe): # adds raw score, kill score, and utility score

    cursor.execute(f"alter table {player_name}_{dataframe} \
                   add column raw_score float")
    cursor.execute(f"alter table {player_name}_{dataframe} \
                add column combat_score float")
    cursor.execute(f"alter table {player_name}_{dataframe} \
                   add column utility_score float")
    cursor.execute(f"update {player_name}_{dataframe} \
                   set raw_score = {player_name}_score - (100 * (atk_wins + def_wins))")
    cursor.execute(f"update {player_name}_{dataframe} \
                   set combat_score = ({player_name}_kills * 100) + ({player_name}_assists * 75)")
    cursor.execute(f"update {player_name}_{dataframe} \
                   set utility_score = {player_name}_score - combat_score")
    
    return

In [9]:
# future-proof way to get all players from the dataframes
players = []
column_names = list(everything.columns)
for column in column_names:
    if "_score" in column:
        players.append(column.replace("_score", ""))

dataframes = ["season", "everything"]

for p in players:
    for df in dataframes:
        get_player_data(p, df)
        extract_score_data(p, df)

In [10]:
# pull the new tables back to python

player_tables = {f"{p}_{df}": pd.read_sql(f"select * from {p}_{df}", conn) for p in players for df in dataframes}

# shorthand, also leaves original dataframe in the dictionary in case i ruin these
josh_season, josh_everything = player_tables["josh_season"], player_tables["josh_everything"]
asa_season, asa_everything = player_tables["asa_season"], player_tables["asa_everything"]
biggs_season, biggs_everything = player_tables["biggs_season"], player_tables["biggs_everything"]
clark_season, clark_everything = player_tables["clark_season"], player_tables["clark_everything"]
cole_season, cole_everything = player_tables["cole_season"], player_tables["cole_everything"]
conell_season, conell_everything = player_tables["conell_season"], player_tables["conell_everything"]
ed_season, ed_everything = player_tables["ed_season"], player_tables["ed_everything"]
jakob_season, jakob_everything = player_tables["jakob_season"], player_tables["jakob_everything"]
luke_season, luke_everything = player_tables["luke_season"], player_tables["luke_everything"]
mike_season, mike_everything = player_tables["mike_season"], player_tables["mike_everything"]
p1_season, p1_everything = player_tables["p1_season"], player_tables["p1_everything"]
p2_season, p2_everything = player_tables["p2_season"], player_tables["p2_everything"]
p3_season, p3_everything = player_tables["p3_season"], player_tables["p3_everything"]
p4_season, p4_everything = player_tables["p4_season"], player_tables["p4_everything"]
p5_season, p5_everything = player_tables["p5_season"], player_tables["p5_everything"]

In [11]:
# additional data cleanup

everything_columns = []
for p in players:
    if f"{p}_score" in list(everything.columns):
        everything_columns.append(f"{p}_score")
        everything_columns.append(f"{p}_kills")
        everything_columns.append(f"{p}_deaths")
        everything_columns.append(f"{p}_assists")

everything = everything.drop(everything_columns, axis = 1)
everything = everything.dropna()

season_columns = []
for p in players:
    if f"{p}_score" in list(season.columns):
        season_columns.append(f"{p}_score")
        season_columns.append(f"{p}_kills")
        season_columns.append(f"{p}_deaths")
        season_columns.append(f"{p}_assists")

season = season.drop(season_columns, axis = 1)
season = season.dropna()

# these removed columns have lots of empty entries that will ruin the tables if we dropna() before dropping these columns,
# and i don't need them anymore since they're in separate tables

In [12]:
# lists for all maps and squad sizes to iterate through
maps = sorted(list(everything["map"].unique()))
sizes = sorted(list(everything["squad_size"].unique()))

In [13]:
# overarching statistics for use in tables
season_wins = len(season[(season["result"] == "win")])
season_losses = len(season[(season["result"] == "loss")])
season_wl = season_wins/season_losses
season_games = len(season)

everything_wins = len(everything[(everything["result"] == "win")])
everything_losses = len(everything[(everything["result"] == "loss")])
everything_wl = everything_wins/everything_losses
everything_games = len(everything)

In [14]:
def win_loss_map(map_name):

    global season_map_wl, season_map_games, everything_map_wl, everything_map_games

    season_filtered = season[season["map"] == map_name]
    season_map_wins = len(season_filtered[(season_filtered["result"] == "win")])
    season_map_losses = len(season_filtered[(season_filtered["result"] == "loss")])
    season_map_games = len(season_filtered)
    if season_map_losses == 0:
        season_map_wl = season_map_wins
    else:
        season_map_wl = season_map_wins/season_map_losses

    everything_filtered = everything[everything["map"] == map_name]
    everything_map_wins = len(everything_filtered[(everything_filtered["result"] == "win")])
    everything_map_losses = len(everything_filtered[(everything_filtered["result"] == "loss")])
    everything_map_games = len(everything_filtered)
    if everything_map_losses == 0:
        everything_map_wl = everything_map_wins
    else:
        everything_map_wl = everything_map_wins/everything_map_losses

    return season_map_wl, season_map_games, everything_map_wl, everything_map_games

In [15]:
def win_loss_squad(squad_size):

    global season_squad_wl, season_squad_games, everything_squad_wl, everything_squad_games

    season_filtered = season[season["squad_size"] == squad_size]
    season_squad_wins = len(season_filtered[(season_filtered["result"] == "win")])
    season_squad_losses = len(season_filtered[(season_filtered["result"] == "loss")])
    season_squad_games = len(season_filtered)

    if season_squad_losses == 0:
        season_squad_wl = season_squad_wins
    else:
        season_squad_wl = season_squad_wins/season_squad_losses

    everything_filtered = everything[everything["squad_size"] == squad_size]
    everything_squad_wins = len(everything_filtered[(everything_filtered["result"] == "win")])
    everything_squad_losses = len(everything_filtered[(everything_filtered["result"] == "loss")])
    everything_squad_games = len(everything_filtered)

    if everything_squad_losses == 0:
        everything_squad_wl = everything_squad_wins
    else:
        everything_squad_wl = everything_squad_wins/everything_squad_losses

    return season_squad_wl, season_squad_games, everything_squad_wl, everything_squad_games

In [16]:
# building a win/loss table for maps

wl_maps = ["Average W/L"]
season_wl_ratios = [season_wl]
season_wl_count = ["na"]
everything_wl_ratios = [everything_wl]
everything_wl_count = ["na"]
# initial inputs set the first row of my final tables

for m in maps:
    win_loss_map(m)
    wl_maps.append(m)
    season_wl_ratios.append(season_map_wl)
    season_wl_count.append(season_map_games)
    everything_wl_ratios.append(everything_map_wl)
    everything_wl_count.append(everything_map_games)

wl_maps_table = pd.DataFrame({"map": wl_maps,
                              "season_wl": season_wl_ratios,
                              "season_games": season_wl_count,
                              "everything_wl": everything_wl_ratios,
                              "everything_games": everything_wl_count})
wl_maps_table.to_csv("C:\\Users\\aethe\\OneDrive\\Desktop\\stats\\y9s2\\wl_maps_y9s2.csv")

In [17]:
# building a win/loss table for squads

wl_squads = ["Average W/L"]
season_wl_ratios = [season_wl]
season_wl_count = ["na"]
everything_wl_ratios = [everything_wl]
everything_wl_count = ["na"]
# initial inputs set the first row of my final tables

for s in sizes:
    win_loss_squad(s)
    wl_squads.append(s)
    season_wl_ratios.append(season_squad_wl)
    season_wl_count.append(season_squad_games)
    everything_wl_ratios.append(everything_squad_wl)
    everything_wl_count.append(everything_squad_games)

wl_squads_table = pd.DataFrame({"size": wl_squads,
                                "season_wl": season_wl_ratios,
                                "season_games": season_wl_count,
                                "everything_wl": everything_wl_ratios,
                                "everything_games": everything_wl_count})
wl_squads_table.to_csv("C:\\Users\\aethe\\OneDrive\\Desktop\\stats\\y9s2\\wl_squads_y9s2.csv")

In [18]:
def performance(p, df): # something is going wrong, fi xit
                        # leaving this note here but im pretty sure its fixed    
    global avg_score, avg_raw, avg_kill_score, avg_util, kd, wl

    d = player_tables[f"{p}_{df}"]
    d = d.dropna()
    
    total_score = d[f"{p}_score"].sum()
    raw_score = d["raw_score"].sum()
    kill_score = d["combat_score"].sum()
    util_score = d["utility_score"].sum()

    games = len(d)

    
    kills = d[f"{p}_kills"].sum()
    deaths = d[f"{p}_deaths"].sum()
    
    wins = len(d[(d["result"] == "win")])
    losses = len(d[(d["result"] == "loss")])

    avg_score = total_score / games if games > 0 else total_score
    avg_kill_score = kill_score / games if games > 0 else kill_score
    avg_raw = raw_score / games if games > 0 else raw_score
    avg_util = util_score / games if games > 0 else util_score
    kd = kills / deaths if deaths > 0 else kills
    wl = wins / losses if losses > 0 else wins

    return avg_score, avg_raw, avg_kill_score, avg_util, kd, wl

In [19]:
perform = pd.DataFrame()
columns = [f"{p}_{df}" for p in players for df in dataframes]

avg_scores = []
avg_raws = []
avg_combats = []
avg_utils = []
kdr = []
wlr = []

for p in players:
    for df in dataframes:
        performance(p, df)
        avg_scores.append(avg_score)
        avg_raws.append(avg_raw)
        avg_combats.append(avg_kill_score)
        avg_utils.append(avg_util)
        kdr.append(kd)
        wlr.append(wl)

perform["Player & Timeframe"] = columns
perform["Score"] = avg_scores
perform["Raw Score"] = avg_raws
perform["Kill Score"] = avg_combats
perform["Utility Score"] = avg_utils
perform["W/L"] = wlr
perform["K/D"] = kdr

perform.to_csv("C:\\Users\\aethe\\OneDrive\\Desktop\\stats\\y9s2\\performance_summary_y9s2.csv")

## Player Data

### Round Data

In [20]:
rounds = pd.read_csv(r"C:\Users\aethe\OneDrive\Desktop\stats\y9s2\y9s2_rounds.csv")
rounds = rounds.drop(["season", "session"], axis = 1)
rounds["map"] = rounds["map"].str.replace(" ", "_")
rounds["map"] = rounds["map"].str.lower()
player_rounds = rounds.drop(["round", "enemy_op1", "enemy_op2", "enemy_op3", "enemy_op4", "enemy_op5"], axis = 1)
cursor.execute("drop table if exists rounds")
player_rounds.to_sql("rounds", conn, index = False)

174

In [21]:
to_remove = ["p1", "p2", "p3", "p4", "p5"] # these representatives are not present in round data

for r in to_remove:
    if r in players:
        players.remove(r)

In [22]:
def get_rounds_data(player): # separate each player into their own table
    cursor.execute(f"drop table if exists {player}_rounds")
    cursor.execute(f"create table {player}_rounds as \
                   select game, map, squad_size, site, side, result, \
                   {player}_op, {player}_kills, {player}_obj, {player}_survived, {player}_traded \
                    from rounds")
    return    

In [23]:
def kost(player): # build and populate kost column
    cursor.execute(f"alter table {player}_rounds add column {player}_kost int")
    cursor.execute(f"update {player}_rounds set {player}_kost = 0 where \
                   {player}_kills = 0 and {player}_obj = 0 and {player}_survived = 0 and {player}_traded = 0")
    cursor.execute(f"update {player}_rounds set {player}_kost = 1 where \
                   {player}_kills = 1 or {player}_obj = 1 or {player}_survived = 1 or {player}_traded = 1")
    return

In [24]:
for p in players:
    get_rounds_data(p)
    kost(p)

In [25]:
indiv_player_rounds = {f"{p}_rounds": pd.read_sql(f"select * from {p}_rounds", conn) for p in players} # pulls tables back into python from sql
sides = ["atk", "def"]

In [26]:
def get_stats(player_name): # gets KOST, W/L, etc.
    
    global kost, win_loss, kill_freq, obj_freq, survival_freq, trade_freq, games

    df = indiv_player_rounds[f"{player_name}_rounds"]
    df = df.dropna()
    games = len(df)
    wins = len(df[(df["result"] == "win")])
    losses = len(df[(df["result"] == "loss")])

    kost = df[f"{player_name}_kost"].sum() / games
    win_loss = wins / losses if losses != 0 else wins
    kill_freq = df[f"{player_name}_kills"].sum() / games
    obj_freq = df[f"{player_name}_obj"].sum() / games
    survival_freq = df[f"{player_name}_survived"].sum() / games
    trade_freq = df[f"{player_name}_traded"].sum() / games

    return kost, win_loss, kill_freq, obj_freq, survival_freq, trade_freq, games

In [27]:
def get_stats_sides(player_name, side): # gets the same stats as above but split into atk and def

    df = indiv_player_rounds[f"{player_name}_rounds"]
    df = df.dropna()

    if side == "atk":
        global atk_kost, atk_win_loss, atk_kill_freq, atk_obj_freq, atk_survival_freq, atk_trade_freq, atk_games

        df = df[df["side"] == "atk"]
        atk_games = len(df)
        wins = len(df[(df["result"] == "win")])
        losses = len(df[(df["result"] == "loss")])

        atk_kost = df[f"{player_name}_kost"].sum() / atk_games
        atk_win_loss = wins / losses if losses != 0 else wins
        atk_kill_freq = df[f"{player_name}_kills"].sum() / atk_games
        atk_obj_freq = df[f"{player_name}_obj"].sum() / atk_games
        atk_survival_freq = df[f"{player_name}_survived"].sum() / atk_games
        atk_trade_freq = df[f"{player_name}_traded"].sum() / atk_games

        return atk_kost, atk_win_loss, atk_kill_freq, atk_obj_freq, atk_survival_freq, atk_trade_freq, atk_games

    if side == "def":
        global def_kost, def_win_loss, def_kill_freq, def_obj_freq, def_survival_freq, def_trade_freq, def_games

        df = df[df["side"] == "def"]
        def_games = len(df)
        wins = len(df[(df["result"] == "win")])
        losses = len(df[(df["result"] == "loss")])

        def_kost = df[f"{player_name}_kost"].sum() / def_games
        def_win_loss = wins / losses if losses != 0 else wins
        def_kill_freq = df[f"{player_name}_kills"].sum() / def_games
        def_obj_freq = df[f"{player_name}_obj"].sum() / def_games
        def_survival_freq = df[f"{player_name}_survived"].sum() / def_games
        def_trade_freq = df[f"{player_name}_traded"].sum() / def_games

        return def_kost, def_win_loss, def_kill_freq, def_obj_freq, def_survival_freq, def_trade_freq, def_games

In [28]:
# apply the data and begin building columns

kosts = []
atk_kosts = []
def_kosts = []
kills = []
atk_kills = []
def_kills = []
obj = []
atk_obj = []
def_obj = []
survivals = []
atk_survivals = []
def_survivals = []
trades = []
atk_trades = []
def_trades = []
count = []
atk_count = []
def_count = []
win_losses = []
atk_win_losses = []
def_win_losses = []

for p in players:
    get_stats(p)
    kosts.append(kost)
    kills.append(kill_freq)
    obj.append(obj_freq)
    survivals.append(survival_freq)
    trades.append(trade_freq)
    count.append(games)
    win_losses.append(win_loss)

for p in players:
    for s in sides:
        if s == "atk":
            get_stats_sides(p, s)

            atk_kosts.append(atk_kost)
            atk_kills.append(atk_kill_freq)
            atk_obj.append(atk_obj_freq)
            atk_survivals.append(atk_survival_freq)
            atk_trades.append(atk_trade_freq)
            atk_count.append(atk_games)
            atk_win_losses.append(atk_win_loss)

        if s == "def":
            get_stats_sides(p, s)
            def_kosts.append(def_kost)
            def_kills.append(def_kill_freq)
            def_obj.append(def_obj_freq)
            def_survivals.append(def_survival_freq)
            def_trades.append(def_trade_freq)
            def_count.append(def_games)
            def_win_losses.append(def_win_loss)

In [29]:
# building the tables

overall_rounds = pd.DataFrame()
overall_rounds["Player"] = players
overall_rounds["Overall W/L Ratio"] = win_losses
overall_rounds["Overall KOST"] = kosts
overall_rounds["Overall Kill Freq"] = kills
overall_rounds["Overall OBJ Freq"] = obj
overall_rounds["Overall Survival Freq"] = survivals
overall_rounds["Overall Trade Freq"] = trades
overall_rounds["Overall Rounds"] = count
overall_rounds.to_csv(r"C:\Users\aethe\OneDrive\Desktop\stats\y9s2\y9s2_players_overall.csv")

atk_rounds = pd.DataFrame()
atk_rounds["Player"] = players
atk_rounds["ATK W/L Ratio"] = atk_win_losses
atk_rounds["ATK KOST"] = atk_kosts
atk_rounds["ATK Kill Freq"] = atk_kills
atk_rounds["ATK OBJ Freq"] = atk_obj
atk_rounds["ATK Survival Freq"] = atk_survivals
atk_rounds["ATK Trade Freq"] = atk_trades
atk_rounds["ATK Rounds"] = atk_count

atk_rounds.to_csv(r"C:\Users\aethe\OneDrive\Desktop\stats\y9s2\y9s2_players_atk.csv")

def_rounds = pd.DataFrame()
def_rounds["Player"] = players
def_rounds["DEF W/L Ratios"] = def_win_losses
def_rounds["DEF KOST"] = def_kosts
def_rounds["DEF Kill Freq"] = def_kills
def_rounds["DEF OBJ Freq"] = def_obj
def_rounds["DEF Survival Freq"] = def_survivals
def_rounds["DEF Trade Freq"] = def_trades
def_rounds["DEF Rounds"] = def_count

def_rounds.to_csv(r"C:\Users\aethe\OneDrive\Desktop\stats\y9s2\y9s2_players_def.csv")

### Maps & Sites

In [30]:
print(maps) # check that everything is fine with the maps

['bank', 'border', 'chalet', 'club_house', 'coastline', 'consulate', 'emerald_plains', 'kafe_dostoyevsky', 'kanal', 'lair', 'nighthaven_labs', 'oregon', 'outback', 'skyscraper', 'stadium_bravo', 'theme_park', 'villa']


In [31]:
def map_sites(map):
    cursor.execute(f"drop table if exists {map}_rounds")
    cursor.execute(f"create table {map}_rounds as \
                   select game, side, site, result \
                   from rounds \
                   where map = '{map}'")

In [32]:
for m in maps:
    map_sites(m)
    print(m)
    print(f"drop table if exists {m}_rounds \
          \ncreate table {m}_rounds as select game, side, site, result from rounds where map = '{m}'")

bank
drop table if exists bank_rounds           
create table bank_rounds as select game, side, site, result from rounds where map = 'bank'
border
drop table if exists border_rounds           
create table border_rounds as select game, side, site, result from rounds where map = 'border'
chalet
drop table if exists chalet_rounds           
create table chalet_rounds as select game, side, site, result from rounds where map = 'chalet'
club_house
drop table if exists club_house_rounds           
create table club_house_rounds as select game, side, site, result from rounds where map = 'club_house'
coastline
drop table if exists coastline_rounds           
create table coastline_rounds as select game, side, site, result from rounds where map = 'coastline'
consulate
drop table if exists consulate_rounds           
create table consulate_rounds as select game, side, site, result from rounds where map = 'consulate'
emerald_plains
drop table if exists emerald_plains_rounds           
create tabl

In [33]:
indiv_map_rounds = {f"{m}_rounds": pd.read_sql(f"select * from {m}_rounds", conn) for m in maps} # pulls tables back into python from sql

In [34]:
def get_sites(map):

    global sites

    df = indiv_map_rounds[f"{map}_rounds"]
    sites = df["site"].unique()

    # print(sites)

    return sites

def get_rounds_data_new(map, side, site):

    global win_loss, num_rounds

    df = indiv_map_rounds[f"{map}_rounds"]
    df = df[df["side"] == side]
    df = df[df["site"] == site]

    wins = len(df[df["result"] == "win"])
    losses = len(df[df["result"] == "loss"])
    num_rounds = len(df)

    win_loss = wins / losses if losses != 0 else wins

    # print(map, side, site, win_loss, num_rounds)

    return map, side, site, win_loss, num_rounds

In [35]:
maps_atk = []
sides_atk = []
sites_atk = []
wl_ratios_atk = []
num_rounds_atk = []

maps_def = []
sides_def = []
sites_def = []
wl_ratios_def = []
num_rounds_def = []

for m in maps:
    if len(indiv_map_rounds[f"{m}_rounds"]) != 0:
        get_sites(m)
        for sd in sides:
            for st in sites:
                get_rounds_data_new(m, sd, st)
                if sd == "atk":
                    maps_atk.append(m)
                    sides_atk.append(sd)
                    sites_atk.append(st)
                    wl_ratios_atk.append(win_loss)
                    num_rounds_atk.append(num_rounds)
                elif sd == "def":
                    maps_def.append(m)
                    sides_def.append(sd)
                    sites_def.append(st)
                    wl_ratios_def.append(win_loss)
                    num_rounds_def.append(num_rounds)

In [36]:
attack_data = pd.DataFrame({
    "Map": maps_atk,
    "Side": sides_atk,
    "Site": sites_atk,
    "W/L": wl_ratios_atk,
    "Rounds": num_rounds_atk
})
attack_data.to_csv(r"C:\Users\aethe\OneDrive\Desktop\stats\y9s2\y9s2_atk_rounds.csv")

defense_data = pd.DataFrame({
    "Map": maps_def,
    "Side": sides_def,
    "Site": sites_def,
    "W/L": wl_ratios_def,
    "Rounds": num_rounds_def
})
defense_data.to_csv(r"C:\Users\aethe\OneDrive\Desktop\stats\y9s2\y9s2_def_rounds.csv")